In [2]:
import os.path

## USER PARAMETERS
rootdir = os.path.abspath('/home/guille/Descargas/mpt/')

In [4]:
# Copyright (C) 2018 - Universitat Pompeu Fabra
# Author - Guillermo Jimenez-Perez <guillermo.jim.per@gmail.com>

# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.

# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

## IMPORTS

import os
import os.path
import struct
import numpy


## GENERAL PARAMETERS
SETW_SEPARATION                       = 9

DICOM_HEADER_FIELD_PRECURSOR          = 4
DICOM_HEADER_FIELD_TYPE               = 4

FILE_HEADER_TOTAL_SIZE                = 276
FILE_HEADER_FIXED_BEGINNING           = 4

FILE_HEADER_FILE_SIZE                 = 4
FILE_HEADER_FILE_TYPE                 = 4
FILE_HEADER_ENDING_CODE               = 4

MPT_FILES_COMMON_HEADER_SIZE          = 12
MPT_FILES_NUMBER_OF_TRACKS_1          = 8
MPT_FILES_NUMBER_OF_TRACKS_2          = 9

MPT_FILE_TOTAL_SIZE                   = 5012
MPT_FILE_TRACK_HEADER                 = 8
MPT_FILE_TRACK_HEADER_TRACK_NUMBER_1  = 0
MPT_FILE_TRACK_HEADER_TRACK_NUMBER_2  = 1
MPT_FILE_ECG_TRACK_DATA               = 5000
MPT_FILE_TRACK_ENDING                 = 4

CATHETER_POSITION_HEADER              = 4
CATHETER_SEGMENTS                     = 251
CATHETER_POSITION_TRACK_SIZE          = CATHETER_SEGMENTS*32 # 251 segments of 32 bytes
CATHETER_POSITION_ENDING              = 4

TOTAL_SEGMENT_SIZE                    = 32
TIME_INDEX_SIGNED_LONG                = 4
TIME_INDEX_LONG                       = 4
X_POSITION                            = 4
Y_POSITION                            = 4
Z_POSITION                            = 4
ANGLE_ALPHA                           = 4
ANGLE_BETA                            = 4
ANGLE_THETA                           = 4

ECG_TRACK_DATA_POINTS                 = 2500
R_WAVE_POSITION                       = 2000
R_WAVE_POSITION_10_HZ                 = 200
LAT_CALCULATION_WINDOW                = 150
UNIPOLAR_TRACK                        = 19
BIPOLAR_TRACK                         = 20

# MAY CHANGE FROM FILE TO FILE!!!
TRACK_EQUIVALENCE = dict()

TRACK_EQUIVALENCE[1] = "I"
TRACK_EQUIVALENCE[2] = "II"
TRACK_EQUIVALENCE[3] = "III"
TRACK_EQUIVALENCE[4] = "aVR"
TRACK_EQUIVALENCE[5] = "aVL"
TRACK_EQUIVALENCE[6] = "aVF"
TRACK_EQUIVALENCE[7] = "V1"
TRACK_EQUIVALENCE[8] = "V2"
TRACK_EQUIVALENCE[9] = "V3"
TRACK_EQUIVALENCE[10] = "V4"
TRACK_EQUIVALENCE[11] = "V5"
TRACK_EQUIVALENCE[12] = "V6"
TRACK_EQUIVALENCE[20] = "M1"
TRACK_EQUIVALENCE[21] = "M2"
TRACK_EQUIVALENCE[22] = "M3"
TRACK_EQUIVALENCE[23] = "M4"
TRACK_EQUIVALENCE[40] = "R1"
TRACK_EQUIVALENCE[41] = "R2"
TRACK_EQUIVALENCE[60] = "M1-M2"
TRACK_EQUIVALENCE[61] = "M3-M4"
TRACK_EQUIVALENCE[80] = "R1-R2"


# Iterate over the files in the folder
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        extension = os.path.splitext(file)[1]

        # If the file is a .dicm file
        if extension == '.mpt':
            # Read the binary file 
            with open(os.path.join(subdir, file), "rb") as dicmFile:
                binaryFile = dicmFile.read();
            
            # Get the number of tracks the .mpt file contains. The information is encoded in the header 
            # of the MPT file, in the 8th and 9th bytes, as a little endian short:
            numberOfTracks = struct.unpack('<h',binaryFile[MPT_FILES_NUMBER_OF_TRACKS_1:MPT_FILES_NUMBER_OF_TRACKS_2+1])

            # Create a matrix for storing the point information
            ekgData        = numpy.zeros((numberOfTracks[0],int(MPT_FILE_ECG_TRACK_DATA/2)))
            latData        = numpy.zeros((numberOfTracks[0],1))
            catheter1Data  = numpy.zeros((6,CATHETER_SEGMENTS))
            catheter2Data  = numpy.zeros((6,CATHETER_SEGMENTS))

            trackCodes     = []

            for j in range(0, numberOfTracks[0]):
                # For convenience, we store the individual track into a variable
                trackInitialPos = MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER + j*(MPT_FILE_ECG_TRACK_DATA + MPT_FILE_TRACK_ENDING + MPT_FILE_TRACK_HEADER)
                trackFinalPos   = MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER + j*(MPT_FILE_ECG_TRACK_DATA + MPT_FILE_TRACK_ENDING + MPT_FILE_TRACK_HEADER) + MPT_FILE_ECG_TRACK_DATA
                track           = binaryFile[trackInitialPos:trackFinalPos]

                # Get the track code for its equivalence (format is short little endian)
                trackCodes.append(struct.unpack('<h', binaryFile[MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER_TRACK_NUMBER_1 + j*MPT_FILE_TOTAL_SIZE:MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TRACK_HEADER_TRACK_NUMBER_2 + j*MPT_FILE_TOTAL_SIZE+1])[0])

                # Each track is encoded as 2500 *little endian* signed short values
                trackFormat     = '<' + ''.join(['h' for k in range(0,MPT_FILE_ECG_TRACK_DATA,2)])
                track           = struct.unpack(trackFormat,track)

                # Store the track into the matrix
                ekgData[j,:] = track

                # Calculate LAT for each track
                LAT = R_WAVE_POSITION - LAT_CALCULATION_WINDOW - 1

                for k in range(R_WAVE_POSITION - LAT_CALCULATION_WINDOW - 1, R_WAVE_POSITION + LAT_CALCULATION_WINDOW):
                    print(k)
                    if track[k] > track[LAT]:
                        LAT = k

                # Introduce the LAT value for the track
                latData[j,:] = (LAT - (R_WAVE_POSITION - 1))

            # Retrieve the information of the position of the catheter for this point
            catheter1InfoStartingPosition = MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TOTAL_SIZE*numberOfTracks[0] + CATHETER_POSITION_HEADER
            catheter2InfoStartingPosition = MPT_FILES_COMMON_HEADER_SIZE + MPT_FILE_TOTAL_SIZE*numberOfTracks[0] + CATHETER_POSITION_HEADER + CATHETER_POSITION_TRACK_SIZE + CATHETER_POSITION_HEADER

            for section in range(0, CATHETER_SEGMENTS):
                # The (x,y,z,alpha,beta,gamma) information is decoded as little endian floats
                xPositionCatheter1 = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4])
                yPositionCatheter1 = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION])
                zPositionCatheter1 = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION])
                alphaCatheter1     = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION])
                betaCatheter1      = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA])
                gammaCatheter1     = struct.unpack('<f',binaryFile[catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA:catheter1InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA])

                xPositionCatheter2 = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4])
                yPositionCatheter2 = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION])
                zPositionCatheter2 = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION])
                alphaCatheter2     = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION])
                betaCatheter2      = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA])
                gammaCatheter2     = struct.unpack('<f',binaryFile[catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA:catheter2InfoStartingPosition+TIME_INDEX_SIGNED_LONG+TIME_INDEX_LONG+TOTAL_SEGMENT_SIZE*section+4+X_POSITION+Y_POSITION+Z_POSITION+ANGLE_ALPHA+ANGLE_BETA])

                # Save the information into the data matrix
                catheter1Data[0,section] = xPositionCatheter1[0]
                catheter1Data[1,section] = yPositionCatheter1[0]
                catheter1Data[2,section] = zPositionCatheter1[0]
                catheter1Data[3,section] = alphaCatheter1[0]
                catheter1Data[4,section] = betaCatheter1[0]
                catheter1Data[5,section] = gammaCatheter1[0]

                catheter2Data[0,section] = xPositionCatheter2[0]
                catheter2Data[1,section] = yPositionCatheter2[0]
                catheter2Data[2,section] = zPositionCatheter2[0]
                catheter2Data[3,section] = alphaCatheter2[0]
                catheter2Data[4,section] = betaCatheter2[0]
                catheter2Data[5,section] = gammaCatheter2[0]


            # Write the point data into files
            outputFileNameCatheter1 = os.path.join(subdir, os.path.splitext(file)[0] + '_Catheter1' + '.txt')
            outputFileNameCatheter2 = os.path.join(subdir, os.path.splitext(file)[0] + '_Catheter2' + '.txt')
            outputFileNameEKG       = os.path.join(subdir, os.path.splitext(file)[0] + '_EKG' + '.txt')
            outputFileNameLAT       = os.path.join(subdir, os.path.splitext(file)[0] + '_LAT' + '.txt')

            # Generate the header for the track files
            headerTracks        = '\t'.join([TRACK_EQUIVALENCE.get(j, 'Unknown') for j in trackCodes]) + '\n'

            # Generate the header for the catheter files
            headerCatheter      = 'X\tY\t\Z\tAlpha\tBeta\tGamma\n'

            # Open files to write
            outputFileCatheter1 = open(outputFileNameCatheter1, "w")
            outputFileCatheter2 = open(outputFileNameCatheter2, "w")
            outputFileEKG       = open(outputFileNameEKG, "w")
            outputFileLAT       = open(outputFileNameLAT, "w")

            # Write headers
            outputFileEKG.write(headerTracks)
            outputFileLAT.write(headerTracks)
            outputFileCatheter1.write(headerCatheter)
            outputFileCatheter2.write(headerCatheter)

            # Write information into track-based files
            for j in range(0,ekgData.shape[1]):
                outputFileEKG.write('\t'.join([str(ekgData[k,j]) for k in range(0,ekgData.shape[0])]) + '\n')

            outputFileLAT.write('\t'.join([str(latData[k,0]) for k in range(0,latData.shape[0])]) + '\n')

            # Write information into catheter-based files
            for j in range(0,catheter1Data.shape[1]):
                outputFileCatheter1.write('\t'.join([str(catheter1Data[k,j]) for k in range(0,catheter1Data.shape[0])]) + '\n')
                outputFileCatheter2.write('\t'.join([str(catheter2Data[k,j]) for k in range(0,catheter2Data.shape[0])]) + '\n')

            outputFileCatheter1.close()
            outputFileCatheter2.close()
            outputFileEKG.close()
            outputFileLAT.close()
                    
    

1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048


1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082


2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
